<a href="https://colab.research.google.com/github/chacha86/pythonai/blob/main/%EC%9B%B9%EC%8A%A4%ED%81%AC%EB%9E%A9%ED%95%912.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests

## 브라우저인 척 하기위해 헤더 정보에 브라우저 정보 포함
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
}

## 해당 url로 요청 보내서 html 문서 받아오기
r = requests.get('https://news.naver.com/main/ranking/popularDay.naver', headers=headers)
    
## html 문서 가져오기
html_doc = r.text

## BeautifulSoup 객체에 넘겨 파싱하기(정보를 태그 단위로 쪼개기)
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_doc, 'html.parser')


## css 선택자를 사용하기 위한 함수 =>
### 1. select()  => 결과가 list
### 2. select_one() => 결과가 1개

## 텍스트와 속성값 얻기
### 1. 태그.text
### 2. 태그[key]
###    - key = value

# 1. 언론사 이름과 링크
boxes = soup.select('.rankingnews_box')

journal_list = []

for box in boxes :

  atag = box.select_one('.rankingnews_box_head')
  strong = box.select_one('.rankingnews_name')

  name = strong.text
  link = atag['href']

  journal = {
      'name' : name,
      'link' : link
  }

  journal_list.append(journal)

target = None
for journal in journal_list :
  if journal['name'] == 'YTN' :
    target = journal
    break







In [52]:
r2 = requests.get(target['link'], headers=headers)

html_doc2 = r2.text

## BeautifulSoup 객체에 넘겨 파싱하기(정보를 태그 단위로 쪼개기)
from bs4 import BeautifulSoup
soup2 = BeautifulSoup(html_doc2, 'html.parser')


## 함수로 쪼개라.

## 페이지에서 전체 뉴스 가져오기
def get_total_news(page) : 
  
  total_news_list = []

  groups = soup2.select('.press_ranking_box')
  for group in groups :
    news_list_tmp = get_news_by_group(group)
    total_news_list.extend(news_list_tmp)

  return total_news_list

## 뉴스 그룹에서 뉴스 꺼내오기
def get_news_by_group(group) :
  news_list = []
  thumb_list = group.select('.as_thumb')
  for news in thumb_list :
    atag = news.select_one('a')
    link = atag['href']
    title = atag.select_one('.list_title').text
    
    news = {
        'title' : title,
        'link' : link
    }
    news_list.append(news)

  return news_list


result = get_total_news(soup2)

In [57]:
import json


def save_to_json(filepath, data) :
  with open(filepath,'w') as f :  
    json_str = json.dumps(data, ensure_ascii=False) 
    f.write(json_str)




In [58]:
def load_to_json(filepath) :
  with open(filepath, 'r') as f:
    json_str = f.read()
    return json.loads(json_str)

  


In [59]:
save_to_json('ytn_news2.json', result)

In [ ]:
load_to_json('/content/ytn_news2.json')